<a href="https://colab.research.google.com/github/vdragon95/lol_project/blob/develop/LOL05_data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pwd

'/content'

In [ ]:
import requests
import pandas as pd
import numpy as np
import time

tier = 'grandmaster' # master, challenger

# match_info_queue_revised_data: queue == 420 데이터만
# match_info_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/' + tier + '/' + \
#                 tier + '_02match_info_queue_revised_data.csv', encoding = 'cp949')
'''☆통합된 버전'''
match_info_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/all/02all_match_info_queue_revised_data.csv', encoding = 'cp949')
print('all_match_info_queue_revised_data file successfully loaded!')

all_match_info_queue_revised_data file successfully loaded!


In [ ]:
# 저장 준비
'''원래는 DB에 각 테이블 저장해야되지만 데이터 분석 집중을 위한 편의성을 위해
파일 크기가 크더라도  한 csv 파일에 임시 저장 하기로 함'''

# 저장 해야 될 정보
'''
[팀 데이터] → 생각해 보니 이거는 이미 03데이터 있음. (같은 매치내 블루 팀 레드 팀간 점수 차이 승패 분석 필요)
2. 매치 정보: 매치(gameId) Id
3. 팀 정보: ,,,

[개인 데이터] → 10 개정보 모두가 있는 정보 추림


[소환사별 데이터]
4. 개인 정보: 소환사 이름, 개인 

[개인 데이터] → 생각해 보니 이거는 이미 04데이터 있음. (같은 매치내 블루 팀 레드 팀간 개개인 평균 점수 차이 승패 분석 필요)

[승리 예측 모델]
개인 데이터를 이용한
팀 데이터를 이용한

[챔피언 데이터]
2. 매치 정보: 챔피언 수
4. 개인 정보: 승, 패,  KDA, totalMinionsKilled, goldEarned
'''

'\n[팀 데이터] → 생각해 보니 이거는 이미 03데이터 있음. (같은 매치내 블루 팀 레드 팀간 점수 차이 승패 분석 필요)\n2. 매치 정보: 매치(gameId) Id\n3. 팀 정보: ,,,\n\n[개인 데이터] → 10 개정보 모두가 있는 정보 추림\n\n\n[소환사별 데이터]\n4. 개인 정보: 소환사 이름, 개인 \n\n[개인 데이터] → 생각해 보니 이거는 이미 04데이터 있음. (같은 매치내 블루 팀 레드 팀간 개개인 평균 점수 차이 승패 분석 필요)\n\n[승리 예측 모델]\n개인 데이터를 이용한\n팀 데이터를 이용한\n\n[챔피언 데이터]\n2. 매치 정보: 챔피언 수\n4. 개인 정보: 승, 패,  KDA, totalMinionsKilled, goldEarned\n'

In [ ]:
# # 파일 병합을 위한 [매치 정보]

# # summoner = list(match_info_df['summonerName'])
# # champion = np.array(match_info_df['champion'])
# # matchId = list(match_info_df['gameId'])
# summoner = np.array(match_info_df['summonerName'][:19880])
# champion = np.array(match_info_df['champion'][:19880], dtype=int)
# matchId = np.array(match_info_df['gameId'][:19880], dtype=int)
# print(len(summoner))
# print(len(champion))
# print(len(matchId))
# matchId = np.array(match_info_df['gameId'][:19880], dtype=int)

# matchId_newax = matchId[:, np.newaxis]

In [ ]:
# [팀 데이터]
## 새로운 칼럼 생성 하기

# 조회를 위해 'matchId'칼럼을 추가
# teams_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/' + tier + '/' + \
#                 tier + '_03team_data_merged.csv', encoding = 'cp949', dtype=int)
'''☆통합된 버전'''
# teams_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/all/03all_team_integrated_data.csv', encoding = 'cp949', dtype=int)

teams_merged_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/all/03team_data_merged.csv', encoding = 'cp949', dtype=int)
# matchId = np.array(match_info_df['gameId'], dtype=int)
# matchId_newax = matchId[:, np.newaxis]

# print('03team_data_merged file successfully loaded!')
# # print(teams_df.shape) # (19880, 11)
teams_col = list(teams_merged_df.columns)
# print(teams_col)
# print(type(teams_col))
# teams_col.insert(0, 'matchId')
# teams_arr = teams_df.values

# print(matchId_newax)
# print(teams_df['gameId'])
# print(teams_arr)

# teams_merged_df = pd.DataFrame(np.concatenate([matchId_newax, teams_arr], axis=1), columns = teams_col)
# # teams_merged_df['matchId'] = teams_merged_df['matchId'].astype('int')
# print(teams_merged_df)
# # teams_merged_df.iloc[1] # 계속 부동 소수점으로 고정됨
# # print(teams_merged_df.columns)
teams_diff_arr = np.zeros((int(len(teams_merged_df) / 2), len(teams_merged_df.columns) - 1))

# [team_diff] 팀간 정보 차이 데이터
for i in range(int(len(teams_merged_df) / 2)):
    matches_Id = teams_merged_df.iloc[i][0]
    blue = np.array(teams_merged_df.iloc[i][2:])
    red = np.array(teams_merged_df.iloc[i + int(len(teams_merged_df) / 2)][2:])

    diff = blue - red

    diff = np.insert(diff, 0, matches_Id)
    teams_diff_arr[i] = diff

teams_diff_arr = teams_diff_arr.astype(int) # 부동 소수점 int화 (나머지 데이터 들도 원래 int)

teams_col.remove('teamId')
teams_diff_df = pd.DataFrame(teams_diff_arr, columns = teams_col)
print(teams_diff_df) # win = -1: blue(100) 승, 1: red(200) 승

    
teams_diff_df.to_csv('/content/drive/MyDrive/Colab Notebooks/all/05teams_diff_data.csv',index=False,encoding = 'cp949')
print('team merged data file saved!')

           gameId  win  firstBlood  ...  towerKills  inhibitorKills  baronKills
0      4766618573   -1          -1  ...          -2              -1           0
1      4766647089    1           1  ...           4               1           1
2      4766596747    1          -1  ...           6               1          -1
3      4764301757   -1          -1  ...          -6              -1          -1
4      4764206762   -1          -1  ...          -9              -3           1
...           ...  ...         ...  ...         ...             ...         ...
39746  4834186338    1           1  ...           6               1           0
39747  4834182423   -1           1  ...          -3              -1           1
39748  4834136566    1           1  ...           0               0           0
39749  4833247515    1           1  ...           4               1          -1
39750  4833167182   -1          -1  ...          -4               0           0

[39751 rows x 11 columns]
team merged d

In [ ]:
# [개인 데이터(10명 경기)]
## 새로운 칼럼 생성 하기

# 조회를 위해 'matchId'칼럼을 추가
indis_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/all/04indi_data_merged.csv', encoding = 'cp949')
'''☆통합된 버전'''
# indis_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/all/04all_individual_integrated_data.csv', encoding = 'cp949')

print(indis_df.sort_values(by="participantId", ascending=True).groupby("matchId").head(1))
print(indis_df.iloc[1140:1160])

# print(indis_df.shape) # (45997, 28) # 10개씩 있직 않으므로 10개인 팀만 추려 내야 함
indis_col = list(indis_df.columns)
# print(indis_col)
# print(len(indis_df.columns))
indis_arr = indis_df.values
# print(indis_arr) # kda가 마지막

# [indi_merged] 팀내 개인 데이터
# 1. 먼저 10개 팀으로 이루어진 것만 골라내기
indis_merged_arr = []

match_Id = 0
index_num = 0
list_num = 0
temp_indis_arr = []

for i, indi_arr in enumerate(indis_arr):
    if match_Id == 0:
        match_Id = indi_arr[0] # 연속된 matchId 자료임을 이용하여 처음 나타난 matchId 저장
        temp_indis_arr.append(indi_arr)
    elif match_Id == indi_arr[0]:
        temp_indis_arr.append(indi_arr)
    else:
        if len(temp_indis_arr) == 10:
            for j in range(10):
                indis_merged_arr.append(temp_indis_arr[j])
            index_num += 10
            list_num += 1
        else:
            temp_indis_arr = []
        match_Id = indi_arr[0] # 다시 저장할 match_Id 초기화
        temp_indis_arr.append(indi_arr)
    if i == len(indis_arr) - 1:
        if len(temp_indis_arr) == 10:
            for j in range(10):
                indis_merged_arr.append(temp_indis_arr[j])

indis_merged_df = pd.DataFrame(indis_merged_arr, columns = indis_col)
print(indis_merged_df.sort_values(by="participantId", ascending=True).groupby("matchId").head(1))

# pd.set_option('display.max_columns', 500) # 펼쳐 보기 
print('04indi_data_merged 데이터 수:', indis_df.shape)
print('손실된 데이터 수:', indis_df.shape[0] - indis_merged_df.shape[0])
print('유저 정보가 10개씩 존재하는 대이터 수 데이터 수:', indis_merged_df.shape) # 10의 배수여야함
print()

# 데이터 저장
indis_merged_df.to_csv('/content/drive/MyDrive/Colab Notebooks/all/06indi_ten_data.csv',index=False,encoding = 'cp949')
print('indis data groupby 10 players file saved!')

             matchId  participantId  teamId  ...  wardsPlaced  wardsKilled       kda
0       4.766619e+09              1     100  ...           11           10  2.500000
93213   4.834426e+09              1     100  ...            6           13  3.333333
51499   4.771718e+09              1     100  ...            9            1  0.714286
20844   4.788664e+09              1     100  ...           14            0  1.666667
158821  4.832257e+09              1     100  ...           16            1  2.000000
...              ...            ...     ...  ...          ...          ...       ...
58985   4.668966e+09              5     100  ...            3            7  1.100000
10414   4.808278e+09              5     100  ...            9            0  1.000000
42661   4.850862e+09              5     100  ...           10            5  1.090909
36647   4.650810e+09              5     100  ...            4            2  0.888889
79846   4.817242e+09              5     100  ...            6    

In [ ]:
# 2. 중복 경기 배제 (같은 경기는 오직 한번만)
print(len(indis_merged_df['matchId'])) # 10 명의 정보가 담긴 데이터 
print(len(set(indis_merged_df['matchId']))) # 중복 배제한 경기 데이터 (* 10 해야됨)

exclusive_matches = list(set(indis_merged_df['matchId'].astype(int)))


# print(indis_df.iloc[0:10])
# print(indis_merged_df.sort_values(by="participantId", ascending=True).groupby("matchId").head(1))
first_index = list(indis_merged_df.sort_values(by="participantId", ascending=True).groupby("matchId").head(1).index)
print(first_index)

exclusive_matches_df = pd.DataFrame()
for i in range(len(first_index)):
    exclusive_matches_df = pd.concat([exclusive_matches_df,indis_merged_df.iloc[first_index[i]:first_index[i]+10]], ignore_index=True)
print(exclusive_matches_df)

# 데이터 저장
exclusive_matches_df.to_csv('/content/drive/MyDrive/Colab Notebooks/all/07indi_exclusive_matches_df.csv',index=False,encoding = 'cp949')
print('indis exclusive_matches_df groupby 10 players file saved!')

84240
4811
[0, 30520, 30530, 30540, 30550, 30560, 30570, 81180, 30580, 30590, 30600, 30610, 30620, 30630, 30640, 30650, 30660, 30670, 81170, 30680, 30690, 30700, 30710, 30720, 30730, 30740, 30510, 30500, 30490, 30480, 30250, 30260, 30270, 81210, 30280, 30290, 30300, 30310, 30320, 30330, 30340, 30350, 30750, 30360, 81200, 30380, 30390, 30400, 30410, 30420, 30430, 30440, 30450, 30460, 30470, 81190, 30370, 30240, 30760, 81160, 31060, 31070, 31080, 31090, 31100, 31110, 31120, 31130, 31140, 31150, 31160, 31170, 81120, 31180, 31190, 31200, 31210, 31220, 31230, 31240, 31250, 31260, 31270, 81110, 31050, 31040, 31030, 31020, 30780, 30790, 30800, 30810, 30820, 30830, 30850, 30860, 30870, 81150, 30880, 30770, 30890, 30910, 30920, 30930, 30940, 30950, 30960, 30970, 81140, 30980, 30990, 31000, 31010, 30900, 30230, 30220, 29440, 29450, 29460, 29470, 81290, 29480, 29490, 29500, 29510, 29520, 29530, 29540, 29550, 29560, 29570, 81280, 29580, 29590, 29600, 29610, 29620, 29630, 29640, 29650, 29660, 29430

In [ ]:
# [소환사명 그룹화 데이터] - 10명 있는 매치 정보만 모아
#                          - 나중에 합칠 정보: Most 3 (픽률, 승, 패,  KDA)

# 결과적으로 소환사별 한명 평균 정보 데이터 (10명게임 아니어도 됨)
#            경기 승패 분석 데이터 (10명 게임만)

pd.set_option('display.max_columns', 500) # 펼쳐 보기 

# match_three_info_df = match_info_df[['summonerName', 'gameId', 'champion']]
indis_col = list(indis_df.columns)
# print(indis_df) → 10명 게임이 없을 수도
# print(indis_merged_df) → 10명인 게임만 있음

indis_col.insert(0, 'champion')
indis_col.insert(0, 'gameId')
indis_col.insert(0, 'summonerName')

indis_col.remove('matchId')
indis_col.remove('championId')

print(exclusive_matches_df.columns)


summoner_merged_df = (pd.merge(match_info_df, exclusive_matches_df, how='right',
                             left_on=['gameId','champion'] , right_on=['matchId','championId']).reindex(columns=indis_col))

summoner_merged_df = summoner_merged_df.dropna(axis=0) # 결측치 제거
summoner_merged_df = summoner_merged_df.reset_index(drop=True)
# print(summoner_merged_df) # kda 이미 있음

neces_col_for_add = ['summonerName', 'win', 'kills', 'deaths', 'assists']

neces_col_for_mean = indis_col
for col in ['gameId', 'champion', 'participantId', 'teamId', 'win']: # win은 따로
    neces_col_for_mean.remove(col)


summoner_sum = summoner_merged_df.loc[:, neces_col_for_add].groupby('summonerName', as_index=False).sum() # kill, deaths, assist 합계

summoner_count = summoner_merged_df.loc[:, ['summonerName','gameId']].groupby('summonerName', as_index=False)['gameId'].count() # Most Top 합계
summoner_count.columns = ['summonerName','count']

# 승률 
summoner_sum['winRate'] = summoner_sum['win'] / summoner_count['count']
print(summoner_sum.columns)

col_merged = list(summoner_count.columns)
col_removed = list(summoner_sum.columns)
col_removed.remove('summonerName')
col_merged.extend(col_removed)
for col in ['kills', 'deaths', 'assists']:
    col_merged.remove(col)
summoner_merged_df1 = (pd.merge(summoner_count, summoner_sum, # how='right',
                             left_on=['summonerName'] , right_on=['summonerName']).reindex(columns=col_merged))

# print(summoner_merged_df1)

summoner_mean = summoner_merged_df.loc[:, neces_col_for_mean].groupby('summonerName', as_index=False).mean() # 평균 통계량

# print('mean_data:\n', summoner_mean) # → for one-line

col_all = list(summoner_merged_df1.columns)
# print(set(summoner_merged_df1.columns) & (set(summoner_mean.columns)))
col_del = list(summoner_mean.columns)
col_del.remove('summonerName')
col_all.extend(col_del)
for col in ['kills', 'deaths', 'assists']:
    col_all.remove(col) # 앞에서부터 지워짐

# summoner_merged_df['winRate'] = summoner_merged_df['win'] / summoner_merged_df['count'].count()

summoner_final_df = (pd.merge(summoner_merged_df1, summoner_mean, # how='right',
                             left_on=['summonerName'] , right_on=['summonerName']).reindex(columns=col_all))
print(summoner_final_df)

# 데이터 저장
summoner_final_df.to_csv('/content/drive/MyDrive/Colab Notebooks/all/08summoner_final_for_analysis.csv',index=False,encoding = 'cp949')
print('summoner_final data file saved!')

Index(['matchId', 'participantId', 'teamId', 'championId', 'win', 'kills',
       'deaths', 'assists', 'totalDamageDealt', 'damageDealtToObjectives',
       'totalDamageDealtToChampions', 'damageDealtToTurrets', 'inhibitorKills',
       'firstTowerKill', 'turretKills', 'goldEarned', 'goldSpent',
       'champLevel', 'totalMinionsKilled', 'firstBloodKill',
       'firstInhibitorKill', 'neutralMinionsKilledTeamJungle',
       'neutralMinionsKilledEnemyJungle', 'neutralMinionsKilled',
       'visionScore', 'wardsPlaced', 'wardsKilled', 'kda'],
      dtype='object')
Index(['summonerName', 'win', 'kills', 'deaths', 'assists', 'winRate'], dtype='object')
    summonerName  count   win   winRate  totalDamageDealt  \
0       DWG jool     30  13.0  0.433333     111882.866667   
1      Gen G Bdd     23  13.0  0.565217     108771.086957   
2     SEKAI ICHI     14  10.0  0.714286     127623.857143   
3         maerd1     12   8.0  0.666667     128044.666667   
4         다드몬아이다     24  16.0  0.66666

In [ ]:
# Most 3
# from collections import Counter # Counter 객체: list 요소별 개수 dict return하는 hashable object (내림차순 자동 정렬)
# champion_dict = Counter(champion)
# print(match_info_df.loc[:, ['champion']]['champion'].value_counts())
champ_df = pd.DataFrame(match_info_df.loc[:, ['champion']]['champion'].value_counts()).reset_index()
champ_df.columns = ['champion','count']
# print(champ_df)

# print(indis_df.iloc[:, 3:])
indis_for_champ = indis_df[['championId', 'win', 'kills', 'deaths', 'assists', 'totalMinionsKilled', 'goldEarned']]
# print(indis_for_champ)
indis_for_champ_col = list(indis_for_champ.columns)
indis_for_champ_col.insert(1, 'count')
print(indis_col) # 'championId', 'win', 'kills', 'deaths', 'assists', 'totalMinionsKilled', 'goldEarned'
champ_merged_df = (pd.merge(champ_df, indis_for_champ, # how='right',
                             left_on=['champion'] , right_on=['championId']).reindex(columns=indis_for_champ_col))

champ_col_for_sum = ['championId', 'win', 'kills', 'deaths', 'assists']
champ_col_for_mean = ['championId', 'totalMinionsKilled', 'goldEarned']
champ_sum = indis_for_champ.loc[:, champ_col_for_sum].groupby('championId', as_index=False).sum() # KDA 승률 → 계산 필요
champ_mean = indis_for_champ.loc[:, champ_col_for_mean].groupby('championId', as_index=False).mean() # CS 처치수, 골드 획득량
champ_match_count = champ_merged_df.loc[:, ['count', 'championId']].groupby('championId', as_index=False).count()
print(champ_sum) # kill, deaths, assist 합계 → KDA
print(champ_mean) # 평균 통계량
print(champ_match_count) # Most 챔프 count
champ_col_for_mean.insert(1,'count')
champ_fin_df = (pd.merge(champ_match_count, champ_sum, # how='right',
                             left_on=['championId'] , right_on=['championId']).reindex(columns=champ_col_for_mean.insert(1,'count')))
champ_fin_df['winRate'] = champ_fin_df['win'] / champ_fin_df['count']
champ_fin_df['kda'] = champ_fin_df[['kills', 'assists']].sum(axis=1) / champ_fin_df['deaths']
# print(champ_fin_df)
# champ_fin_df.columns.append('totalMinionsKilled', 'goldEarned')
champ_fin_col = list(champ_fin_df.columns)
champ_final_df = (pd.merge(champ_fin_df, champ_mean, # how='right',
                             left_on=['championId'] , right_on=['championId']).reindex(columns=champ_fin_col.extend(['totalMinionsKilled', 'goldEarned'])))
# print(champ_final_df)

champ_final_df = champ_final_df.sort_values(by=['count'], axis=0, ascending=False, ignore_index=True)
print(champ_final_df)


# 데이터 저장
champ_final_df.to_csv('/content/drive/MyDrive/Colab Notebooks/all/09champion_statistic.csv', index=False, encoding = 'cp949')
print('champion data groupby 10 players file saved!')


['summonerName', 'kills', 'deaths', 'assists', 'totalDamageDealt', 'damageDealtToObjectives', 'totalDamageDealtToChampions', 'damageDealtToTurrets', 'inhibitorKills', 'firstTowerKill', 'turretKills', 'goldEarned', 'goldSpent', 'champLevel', 'totalMinionsKilled', 'firstBloodKill', 'firstInhibitorKill', 'neutralMinionsKilledTeamJungle', 'neutralMinionsKilledEnemyJungle', 'neutralMinionsKilled', 'visionScore', 'wardsPlaced', 'wardsKilled', 'kda']
     championId   win  kills  deaths  assists
0             1    69    519     831     1507
1             2   895  12608    8648    12384
2             3  1754  13143   14633    33618
3             4   431   3846    3996     7257
4             5   214   2767    2806     3851
..          ...   ...    ...     ...      ...
148         526   104    447    1586     2750
149         555   551   6976    7708     9966
150         777   624   7534    7671     7557
151         875  2346  16625   28203    45902
152         876  2201  28606   20725    39461
